# Lab | Data Aggregation and Filtering

In this challenge, we will continue to work with customer data from an insurance company. We will use the dataset called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by first performing data cleaning, formatting, and structuring.

2. Using the original Dataframe, analyze:
   - the average `monthly_premium` and/or customer lifetime value by `policy_type` and `gender` for customers who responded "Yes", and
   - compare these insights to `total_claim_amount` patterns, and discuss which segments appear most profitable or low-risk for the company.

3. Analyze the total number of customers who have policies in each state, and then filter the results to only include states where there are more than 500 customers.

4. Find the maximum, minimum, and median customer lifetime value by education level and gender. Write your conclusions.

## Bonus

5. The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.

6.  Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

*Hint:*
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

7. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

External Resources for Data Filtering: https://towardsdatascience.com/filtering-data-frames-in-pandas-b570b1f834b9

In [3]:
import pandas as pd

# ---------------------------------------------------
# Load data
# ---------------------------------------------------
url = "https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv"
df = pd.read_csv(url)

# ---------------------------------------------------
# Clean column names
# ---------------------------------------------------
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')

# ---------------------------------------------------
# 1. Average monthly premium & CLV by policy type and gender
#    (Only customers who responded "Yes")
# ---------------------------------------------------
yes_response_df = df[df['response'] == 'Yes']

avg_premium_clv = (
    yes_response_df
    .groupby(['policy_type', 'gender'])
    .agg(
        avg_monthly_premium=('monthly_premium_auto', 'mean'),
        avg_clv=('customer_lifetime_value', 'mean'),
        avg_total_claim=('total_claim_amount', 'mean')
    )
    .round(2)
)

print("Average Premium, CLV, and Claims (Yes Responders):")
display(avg_premium_clv)

# ---------------------------------------------------
# 2. Total number of customers per state (> 500 customers)
# ---------------------------------------------------
customers_by_state = (
    df.groupby('state')
    .size()
    .reset_index(name='customer_count')
)

states_over_500 = customers_by_state[customers_by_state['customer_count'] > 500]

print("States with more than 500 customers:")
display(states_over_500)

# ---------------------------------------------------
# 3. Max, Min, Median CLV by education level and gender
# ---------------------------------------------------
clv_stats = (
    df.groupby(['education', 'gender'])
    .agg(
        max_clv=('customer_lifetime_value', 'max'),
        min_clv=('customer_lifetime_value', 'min'),
        median_clv=('customer_lifetime_value', 'median')
    )
    .round(2)
)

print("Customer Lifetime Value stats by Education and Gender:")
display(clv_stats)

# ---------------------------------------------------
# BONUS PART 1:
# Number of policies sold by state and month
# ---------------------------------------------------
policies_by_state_month = (
    df.groupby(['state', 'month'])
    .size()
    .reset_index(name='policies_sold')
)

# ---------------------------------------------------
# BONUS PART 2:
# Top 3 states with highest number of policies sold
# ---------------------------------------------------
top_states = (
    policies_by_state_month
    .groupby('state')['policies_sold']
    .sum()
    .sort_values(ascending=False)
    .head(3)
    .index
)

# ---------------------------------------------------
# BONUS PART 3:
# Pivot table (months as columns, states as rows)
# ---------------------------------------------------
top_states_pivot = (
    policies_by_state_month
    [policies_by_state_month['state'].isin(top_states)]
    .pivot(index='state', columns='month', values='policies_sold')
    .fillna(0)
    .astype(int)
)

print("Policies sold by Month for Top 3 States:")
display(top_states_pivot)

# ---------------------------------------------------
# BONUS PART 4:
# Marketing channel impact on response rate
# ---------------------------------------------------
marketing_cols = ['marketing_channel_1', 'marketing_channel_2', 'marketing_channel_3']

marketing_melted = df.melt(
    id_vars=['response'],
    value_vars=marketing_cols,
    var_name='marketing_channel',
    value_name='channel_value'
)

response_rate_by_channel = (
    marketing_melted
    [marketing_melted['channel_value'] == 'Yes']
    .groupby('marketing_channel')['response']
    .value_counts(normalize=True)
    .rename('response_rate')
    .reset_index()
)

print("Customer response rate by marketing channel:")
display(response_rate_by_channel)


Average Premium, CLV, and Claims (Yes Responders):


avg_monthly_premium  avg_clv  avg_total_claim
policy_type    gender                                               
Corporate Auto F                     94.30  7712.63           433.74
               M                     92.19  7944.47           408.58
Personal Auto  F                     99.00  8339.79           452.97
               M                     91.09  7448.38           457.01
Special Auto   F                     92.31  7691.58           453.28
               M                     86.34  8247.09           429.53

States with more than 500 customers:


,state,customer_count
0,Arizona,1937
1,California,3552
2,Nevada,993
3,Oregon,2909
4,Washington,888


Customer Lifetime Value stats by Education and Gender:


max_clv  min_clv  median_clv
education            gender                               
Bachelor             F       73225.96  1904.00     5640.51
                     M       67907.27  1898.01     5548.03
College              F       61850.19  1898.68     5623.61
                     M       61134.68  1918.12     6005.85
Doctor               F       44856.11  2395.57     5332.46
                     M       32677.34  2267.60     5577.67
High School or Below F       55277.45  2144.92     6039.55
                     M       83325.38  1940.98     6286.73
Master               F       51016.07  2417.78     5729.86
                     M       50568.26  2272.31     5579.10

KeyError: 'month'